In [ ]:
import numpy as np
from collections import Counter, defaultdict
import concurrent.futures

# Define ranks and suits
ranks = np.arange(13)  # Map ranks to integers 0-12 (2 to Ace)
suits = np.arange(4)   # Map suits to integers 0-3 (hearts, diamonds, clubs, spades)

# Create a deck with two identical sets of 52 cards (104 cards total)
deck = np.array([(rank, suit) for rank in ranks for suit in suits] * 2, dtype=[('rank', 'i4'), ('suit', 'i4')])

def print_deck(deck):
    """Print the entire deck to show all cards in both identical sets."""
    for card in deck:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        print(f"{rank_name} of {suit_name}", end=", ")
    print("\n")

def print_hand(hand):
    """Print the hand in a readable 'rank of suit' format."""
    hand_cards = []
    for card in hand:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        hand_cards.append(f"{rank_name} of {suit_name}")
    print("Hand:", ", ".join(hand_cards))

def deal_hand(deck):
    """
    Shuffle the deck and select five unique cards.
    """
    shuffled_deck = np.random.permutation(deck)
    hand_indices = np.random.choice(len(shuffled_deck), 5, replace=False)
    hand = shuffled_deck[hand_indices]
    sorted_hand = np.sort(hand, order='rank')
    return shuffled_deck, sorted_hand

def check_two_pair_case(hand):
    """Classify a Two Pair hand into Case 0, Case 1, or Case 2, and eliminate Flush hands."""
    # Check for Flush: exclude if all cards are of the same suit
    if len(set(card['suit'] for card in hand)) == 1:
        return -1  # This is a Flush, not a valid Two Pair

    # Extract ranks and count occurrences
    rank_counts = Counter(card['rank'] for card in hand)
    suit_counts = Counter((card['rank'], card['suit']) for card in hand)

    # Identify ranks that form pairs
    pairs = [rank for rank, count in rank_counts.items() if count == 2]

    # If not exactly two pairs, it’s not a Two Pair hand
    if len(pairs) != 2:
        return -1

    # Sort pairs by suit for case checks
    first_pair_rank, second_pair_rank = pairs

    # Case 0: Both pairs are distinct in suits, and fifth card is unrelated rank and suit
    if (
    len(set(card['suit'] for card in hand if card['rank'] == first_pair_rank)) == 2 and
    len(set(card['suit'] for card in hand if card['rank'] == second_pair_rank)) == 2 and
    all(card['rank'] != first_pair_rank and card['rank'] != second_pair_rank for card in hand if rank_counts[card['rank']] == 1)):
     return 0

    # Case 1: First pair is identical in both rank and suit; second pair has different suits
    if (
    any(count == 2 for count in suit_counts.values() if count == 2) and
    len(set(card['suit'] for card in hand if card['rank'] == second_pair_rank)) == 2 and
    all(card['rank'] != first_pair_rank and card['rank'] != second_pair_rank for card in hand if rank_counts[card['rank']] == 1)):
     return 1

    if (
    all(count == 2 for count in suit_counts.values() if count == 2) and
    all(card['rank'] != first_pair_rank and card['rank'] != second_pair_rank for card in hand if rank_counts[card['rank']] == 1)):
     return 2

def run_simulation(num_trials, worker_id=None):
    """
    Run simulation with error checking.
    """
    try:
        if worker_id is not None:
            # Create unique seed using large prime numbers and worker id
            base_seed = 104729  # Large prime
            multiplier = 15485863  # Another large prime
            worker_seed = base_seed + (worker_id * multiplier)
            np.random.seed(worker_seed)

        counts = Counter()
        local_examples = defaultdict(list)

        batch_size = 100_000
        num_batches = num_trials // batch_size + (1 if num_trials % batch_size else 0)

        for batch in range(num_batches):
            current_batch_size = min(batch_size, num_trials - batch * batch_size)
            batch_counts = Counter()

            for _ in range(current_batch_size):
                _, hand = deal_hand(deck)
                case = check_two_pair_case(hand)
                if case >= 0:  # Valid cases are 0, 1, 2
                    batch_counts[case] += 1
                    if len(local_examples[case]) < 10:
                        local_examples[case].append(hand.copy())

            counts.update(batch_counts)

            if worker_id is not None:
                print(f"Worker {worker_id}: Completed batch {batch + 1}/{num_batches}, "
                      f"found Case 0: {counts[0]}, "
                      f"Case 1: {counts[1]}, "
                      f"Case 2: {counts[2]} so far")

        return counts, local_examples

    except Exception as e:
        print(f"Error in worker {worker_id}: {str(e)}")
        raise

def run_simulation_parallel(trials, num_workers=16):
    """
    Run parallel simulation with error checking.
    """
    try:
        chunk_size = trials // num_workers
        remaining = trials % num_workers
        total_counts = Counter()
        all_examples = defaultdict(list)

        with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
            futures = []
            for i in range(num_workers):
                worker_trials = chunk_size + (remaining if i == num_workers - 1 else 0)
                futures.append(executor.submit(run_simulation, worker_trials, worker_id=i))

            for future in concurrent.futures.as_completed(futures):
                counts, examples = future.result()
                total_counts.update(counts)

                for case in [0, 1, 2]:
                    if len(all_examples[case]) < 10 and case in examples:
                        needed_examples = 10 - len(all_examples[case])
                        all_examples[case].extend(examples[case][:needed_examples])

        print("\nTotal examples collected:")
        for case in [0, 1, 2]:
            print(f"Case {case}: {len(all_examples[case])} examples")

        return total_counts, all_examples

    except Exception as e:
        print(f"Error in parallel simulation: {str(e)}")
        raise


def calculate_two_pair_probability(two_pair_count, total_hands):
    """Calculate the probability of getting a Two Pair."""
    return two_pair_count / total_hands

def main():
    # Print the initial unshuffled deck
    print("Initial deck:")
    print_deck(deck)

    # Part 1: Run 100 trials, shuffle each time, and print the shuffled deck and dealt hand
    print("\nRunning 100 trials:")
    example_hands_100 = {"Case 0": [], "Case 1": [], "Case 2": []}
    for trial in range(100):
        shuffled_deck, hand = deal_hand(deck)  # Shuffle and deal a hand
        print(f"\nTrial {trial + 1}: Shuffled deck")
        print_deck(shuffled_deck)  # Print the entire shuffled deck after shuffling

        # Print the first 5 cards as the dealt hand
        print_hand(hand)

        # Check for a two pair in the dealt hand
        case = check_two_pair_case(hand)
        print(f"Two Pair Case: {case}")

        # Collect examples during the initial 100 trials if needed
        if case in example_hands_100 and len(example_hands_100[case]) < 10:
            example_hands_100[case].append(hand)

    # Part 2: Run full simulation for larger trial counts and calculate total two pair probability
    trial_counts = [1_00, 1_000, 10_000, 100_000, 1_000_000, 10_000_000, 100_000_000]
    for trials in trial_counts:
        print(f"\nRunning parallel simulation with {trials} trials...")
        counts, example_hands_large = run_simulation_parallel(trials)

        # Calculate total number of two pair hands (sum of all cases)
        total_two_pair_count = sum(counts[case] for case in [0, 1, 2])

        # Calculate total probability for two pair hands
        total_probability = calculate_two_pair_probability(total_two_pair_count, trials)

        # Print results and total probability for two-pair hands
        print(f"Results for {trials} trials: {dict(counts)}")
        print(f"Total probability of two pair in {trials} trials: {total_probability:.8f}")

        # After completing 1,000,000 trials, print 10 example hands for each case
        if trials == 1_000_000:
            print("\nExample Hands from the 1,000,000-trial simulation for each case:")

            # Print all available examples for Case 0
            print("\nCase 0 examples:")
            for hand in example_hands_large[0]:  # Print all hands for Case 0
                print_hand(hand)

            # Print up to 10 examples for Case 1
            print("\n10 Example Hands for Case 1:")
            for hand in example_hands_large[1][:10]:  # Limit to 10 hands for Case 1
                print_hand(hand)

            # Print up to 10 examples for Case 2
            print("\n10 Example Hands for Case 2:")
            for hand in example_hands_large[2][:10]:  # Limit to 10 hands for Case 2
                print_hand(hand)

if __name__ == "__main__":
    main()



Initial deck:
2 of Hearts, 2 of Diamonds, 2 of Clubs, 2 of Spades, 3 of Hearts, 3 of Diamonds, 3 of Clubs, 3 of Spades, 4 of Hearts, 4 of Diamonds, 4 of Clubs, 4 of Spades, 5 of Hearts, 5 of Diamonds, 5 of Clubs, 5 of Spades, 6 of Hearts, 6 of Diamonds, 6 of Clubs, 6 of Spades, 7 of Hearts, 7 of Diamonds, 7 of Clubs, 7 of Spades, 8 of Hearts, 8 of Diamonds, 8 of Clubs, 8 of Spades, 9 of Hearts, 9 of Diamonds, 9 of Clubs, 9 of Spades, 10 of Hearts, 10 of Diamonds, 10 of Clubs, 10 of Spades, J of Hearts, J of Diamonds, J of Clubs, J of Spades, Q of Hearts, Q of Diamonds, Q of Clubs, Q of Spades, K of Hearts, K of Diamonds, K of Clubs, K of Spades, A of Hearts, A of Diamonds, A of Clubs, A of Spades, 2 of Hearts, 2 of Diamonds, 2 of Clubs, 2 of Spades, 3 of Hearts, 3 of Diamonds, 3 of Clubs, 3 of Spades, 4 of Hearts, 4 of Diamonds, 4 of Clubs, 4 of Spades, 5 of Hearts, 5 of Diamonds, 5 of Clubs, 5 of Spades, 6 of Hearts, 6 of Diamonds, 6 of Clubs, 6 of Spades, 7 of Hearts, 7 of Diamonds, 